In [269]:
file = open('shape.map')
lines = file.readlines()

In [270]:
n=0
allent = []
allgeo = []
newent = {'geos':[]}
newgeo = []
for line in lines:
    if line.startswith('{'):
        n+=1
    if line.startswith('}'):
        n-=1
        if n==0:
            allent.append(newent)
            newent = {'geos':[]} 
        if n==1:
            newent['geos'].append(len(allgeo))
            allgeo.append(newgeo)
            newgeo = []
    if n==1 and line.startswith('"'):
        tk=line.split('"')
        newent[tk[1]]=tk[3]
    if n==2 and line.startswith('('):
        tk=line.split()
        newgeo.append(tk)

#print(allent)
#print(allgeo)

In [271]:
import numpy as np

EPS = 1e-9

EQS_ID = 31
POLY_ID = 32
TEXUV_ID = 33
NORM_ID = 34
PVI_ID = 35
PTI_ID = 36
PNI_ID = 37

for geo in allgeo:
    for plane in geo:
        p1=np.array([float(plane[1]), float(plane[2]), float(plane[3])])
        p2=np.array([float(plane[6]), float(plane[7]), float(plane[8])])
        p3=np.array([float(plane[11]),float(plane[12]),float(plane[13])])
        n =np.cross(p1-p3,p2-p3)
        n/=np.linalg.norm(n)
        d =np.dot(-n,p1)
        plane.append([n,d])
        plane.append([])
        plane.append([])
        plane.append(n )
        plane.append([])
        plane.append([])
        plane.append([])
        
def generate_vertices(geo):
    nn = len(geo)
    for i in range(0,nn-2):
        for j in range(i+1,nn-1):
            for k in range(j+1,nn):
                if i==j or i==k or j==k:continue
                n1=geo[i][EQS_ID][0]
                n2=geo[j][EQS_ID][0]
                n3=geo[k][EQS_ID][0]
                d1=geo[i][EQS_ID][1]
                d2=geo[j][EQS_ID][1]
                d3=geo[k][EQS_ID][1]

                dn=np.dot(n1,np.cross(n2,n3))
                if -EPS<dn and dn<EPS:continue

                p=(-d1*(np.cross(n2,n3))-d2*(np.cross(n3,n1))-d3*(np.cross(n1,n2)))/dn

                inside=True
                for m in geo:
                    nm=m[EQS_ID][0]
                    dm=m[EQS_ID][1]
                    if np.dot(nm,p)+dm < 0:
                        inside=False
                        break
                if inside:
                    geo[i][POLY_ID].append(p)
                    geo[j][POLY_ID].append(p)
                    geo[k][POLY_ID].append(p)

for geo in allgeo:
    generate_vertices(geo)


In [272]:
def sort_vertices(poly):
    tmppoly = poly
    n = len(tmppoly)

    center = np.array([0.,0.,0.])
    for i in tmppoly:
        center += i
    nf = float(n)
    if -EPS<nf and nf<EPS:nf=EPS
    center /= nf

    for i in range(n-2):
        a = tmppoly[i] - center
        z = np.linalg.norm(a)
        if -EPS<z and z<EPS:z=EPS
        a /= z

        nm = np.cross(plane[EQS_ID][0],a)

        smallestAngle = -1.0
        smallest = -1

        for j in range(i+1, n):
            if np.dot(tmppoly[j] - center, nm) < 0:
                b = tmppoly[j] - center
                nz = np.linalg.norm(b)
                if -EPS<nz and nz<EPS:nz=EPS
                b /= nz
                angle = np.dot(a,b)

                if angle > smallestAngle:
                    smallestAngle = angle
                    smallest = j

        tmp = tmppoly[i+1]
        tmppoly[i+1]=tmppoly[smallest]
        tmppoly[smallest] = tmp

    return tmppoly

for geo in allgeo:
    for plane in geo:
        plane[POLY_ID] = sort_vertices(plane[POLY_ID])
       

        

In [273]:
def generate_uv(plane, vertex):
    texname = plane[15]
    ubasis = np.array([float(plane[17]),float(plane[18]),float(plane[19])])
    vbasis = np.array([float(plane[23]),float(plane[24]),float(plane[25])])
    uoff   = float(plane[20])
    voff   = float(plane[26])

    w = 32
    h = 32

    ro = float(plane[28])
    su = float(plane[29])
    sv = float(plane[30])

    tu = (np.dot(vertex,ubasis)/su + uoff) / w
    tv = (np.dot(vertex,vbasis)/sv + voff) / h

    return [tu - int(tu),tv - int(tv)]

for geo in allgeo:
    for plane in geo:
        for vertex in plane[POLY_ID]:
            plane[TEXUV_ID].append(generate_uv(plane, vertex))

o = open('vt.obj', 'w')
for geo in allgeo:
    for plane in geo:
        #for i in range(len(plane)):
        #    print(f'{i}\t{plane[i]}')
        for v in plane[POLY_ID]:
            o.write(f'v {v[0]} {v[1]} {v[2]}\n')

n = 1
for geo in allgeo:
    for plane in geo:
        o.write(f'f {n} {n+1} {n+2}\n')
        o.write(f'f {n} {n+2} {n+3}\n')
        n+=4

In [ ]:
def clipToBrush(cbrush, brush, clipOnPlane):
    tcbrush = cbrush
    nn = len(cbrush)
    for i in range(nn):
        tcbrush[]

def csg_union(geolist):
    c_geolist = geolist

    clipOnPlane = False

    nn = len(c_geolist)
    for i in range(n):
        clipOnPlane = False
        for j in range(n):
            if not i==j:
                clipToBrush(c_geolist[i], geolist[j], clipOnPlane)
            else:
                clipOnPlane = True

    return c_geolist
    
allgeo = csg_union(allgeo)